In [1]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *

In [2]:
import pickle
with open("data/parsed/ml-1m-new", 'rb') as f:
    (tr_users, val_users, te_users, train_data, val_tr, val_te, te_tr, te_te) = pickle.load(f)
empty = csr_matrix(train_data.shape)

In [3]:
n_users, n_items = train_data.shape
ae_dataset = AEDataset(train_data[tr_users])
total_anneal_steps = 50000
anneal_cap = 0.2


In [4]:
vin = val_tr[val_users]
vo = val_te[val_users]

In [5]:
batch_size = 128

In [8]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)
use_vae=True
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([200] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([200] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5 * 1e-4)
sc = neuralsort.neuralobjs.SC()
lm = -1
update_count = 0
for epoch in (range(1, 500 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar)
            update_count += 1
        else:
            scores = model.forward(row)
            loss  = models.loss.MultinomialLoss(row, scores)
            
        (loss.mean()).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 25 == 0):
        tr_loss = torch.cat(tr_losses).mean()

        model.eval()
        model.training=False
        wrapper1 = models.ae.implicitWrapper(model, train_data, naive_sparse2tensor, vae=True)
        wrapper2 = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)
        tr_ndcg = (ranking_metrics_at_k(wrapper1, empty, train_data, K=5, num_threads=4)['map'])
        val_ndcg = (ranking_metrics_at_k(wrapper2, vin, vo, K=5, num_threads=4)['map'])

        ret = (tr_loss.detach().cpu().numpy(), tr_ndcg, val_ndcg)
        print(["%0.4f" % x for x in ret])
        qed.append(ret)
no_add = qed

['266.5675', '0.7016', '0.2832']
['261.3681', '0.7340', '0.3041']
['261.8279', '0.7524', '0.3088']
['260.2758', '0.7639', '0.3088']
['259.7892', '0.7671', '0.3070']
['257.2447', '0.7792', '0.3141']
['256.4285', '0.7788', '0.3117']
['254.3275', '0.7789', '0.3121']
['253.5939', '0.7799', '0.3052']
['254.6119', '0.7742', '0.3104']
['253.1598', '0.7708', '0.3081']
['251.7246', '0.7705', '0.3056']
['252.1712', '0.7684', '0.3072']
['251.8128', '0.7716', '0.3064']
['250.6313', '0.7693', '0.3077']
['251.1596', '0.7724', '0.3031']
['250.2956', '0.7709', '0.3061']
['250.6282', '0.7716', '0.3058']
['251.0723', '0.7739', '0.3047']
['250.8794', '0.7736', '0.3058']


In [9]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)
use_vae=True
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([200] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([200] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5 * 1e-4)
sc = neuralsort.neuralobjs.SC()
lm = -1 

update_count = 0
for epoch in (range(1, 500 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar)
            update_count += 1

        else:
            scores = model.forward(row)
            loss  = models.loss.MultinomialLoss(row, scores)

        loss2 = neuPrecLoss(sc, scores, row, topk=100, k=3, tau=15.0, use_top=True)
        (loss.mean() + 30.0 * loss2.mean()).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 25 == 0):
        #vind = torch.FloatTensor(np.asarray(vin.todense()).astype(np.float32)).cuda()
        tr_loss = torch.cat(tr_losses).mean()

        model.eval()
        model.training=False
        wrapper1 = models.ae.implicitWrapper(model, train_data, naive_sparse2tensor, vae=True)
        wrapper2 = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)
        tr_ndcg = (ranking_metrics_at_k(wrapper1, empty, train_data, K=5, num_threads=4)['map'])
        val_ndcg = (ranking_metrics_at_k(wrapper2, vin, vo, K=5, num_threads=4)['map'])

        ret = (tr_loss.detach().cpu().numpy(), tr_ndcg, val_ndcg)
        print(["%0.4f" % x for x in ret])
        qed.append(ret)
add = qed

['267.2704', '0.7743', '0.2899']
['263.7904', '0.8155', '0.3108']
['261.8874', '0.8376', '0.3187']
['260.2988', '0.8458', '0.3285']
['260.6927', '0.8588', '0.3278']
['259.2096', '0.8630', '0.3303']
['258.0349', '0.8649', '0.3254']
['257.0943', '0.8645', '0.3260']
['255.5931', '0.8669', '0.3258']
['256.7205', '0.8649', '0.3245']
['253.9212', '0.8626', '0.3222']
['254.2529', '0.8613', '0.3194']
['254.1992', '0.8585', '0.3198']
['253.3537', '0.8567', '0.3197']
['252.1020', '0.8520', '0.3163']
['252.4483', '0.8529', '0.3133']
['252.4604', '0.8517', '0.3202']
['252.8360', '0.8519', '0.3164']
['252.8104', '0.8545', '0.3167']
['251.8477', '0.8549', '0.3145']


In [ ]:
add

In [ ]:
no_add